## [教學目標]
學習 sklearn 中，各種評估指標的使用與意義

## [範例重點]
注意觀察各指標的數值範圍，以及輸入函數中的資料格式

In [1]:
from sklearn import metrics, datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import numpy as np
%matplotlib inline

## 回歸問題
常見的評估指標有
- MAE
- MSE
- R-square

我們隨機生成(X, y)資料，然後使用線性回歸模型做預測，再使用 MAE, MSE, R-square 評估

In [2]:
X, y = datasets.make_regression(n_features=1, random_state=42, noise=4) # 生成資料
model = LinearRegression() # 建立回歸模型
model.fit(X, y) # 將資料放進模型訓練
prediction = model.predict(X) # 進行預測
mae = metrics.mean_absolute_error(prediction, y) # 使用 MAE 評估
mse = metrics.mean_squared_error(prediction, y) # 使用 MSE 評估
r2 = metrics.r2_score(prediction, y) # 使用 r-square 評估
print("MAE: ", mae)
print("MSE: ", mse)
print("R-square: ", r2)

MAE:  2.841797252565566
MSE:  12.48868006739824
R-square:  0.9916581036260311


## 分類問題
常見的評估指標有
- AUC
- F1-Score (Precision, Recall)

In [3]:
cancer = datasets.load_breast_cancer() # 我們使用 sklearn 內含的乳癌資料集
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=50, random_state=0)

In [4]:
print(y_test) # 測試集中的 label

[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 0 1 0 1 1 0 1 1 1 0]


In [5]:
y_pred = np.random.random((50,)) # 我們先隨機生成 50 筆預測值，範圍都在 0~1 之間，代表機率值

In [6]:
print(y_pred)

[0.94436707 0.96981716 0.61044511 0.16130643 0.14125129 0.36113079
 0.16930552 0.63543835 0.91484354 0.04232795 0.30149045 0.27400938
 0.88130454 0.63332724 0.28281732 0.37100792 0.10230885 0.49327476
 0.22577428 0.0845378  0.30502409 0.80584087 0.77549302 0.9296871
 0.04683726 0.4877093  0.5375602  0.01500851 0.22186233 0.95885252
 0.39840607 0.07930916 0.85143615 0.79634753 0.12078251 0.25658046
 0.45545786 0.27298705 0.34894544 0.65497088 0.56570529 0.23182799
 0.24334896 0.93752906 0.53767368 0.36876289 0.09804955 0.95026601
 0.10533739 0.6436481 ]


### AUC

In [7]:
auc = metrics.roc_auc_score(y_test, y_pred) # 使用 roc_auc_score 來評估。 **這邊特別注意 y_pred 必須要放機率值進去!**
print("AUC: ", auc) # 得到結果約 0.5，與亂猜的結果相近，因為我們的預測值是用隨機生成的

AUC:  0.5314091680814941


## F1-Score

In [19]:
threshold = 0.5 
y_pred_binarized = np.where(y_pred>threshold, 1, 0) # 使用 np.where 函數, 將 y_pred > 0.5 的值變為 1，小於 0.5 的為 0
f1 = metrics.f1_score(y_test, y_pred_binarized) # 使用 F1-Score 評估
precision = metrics.precision_score(y_test, y_pred_binarized) # 使用 Precision 評估
recall  = metrics.recall_score(y_test, y_pred_binarized) # 使用 recall 評估
print("F1-Score: ", f1) 
print("Precision: ", precision)
print("Recall: ", recall)

F1-Score:  0.5098039215686274
Precision:  0.65
Recall:  0.41935483870967744


In [23]:
metrics.confusion_matrix(y_test, y_pred_binarized)

array([[12,  7],
       [18, 13]], dtype=int64)

In [26]:
13/20

0.65

## [作業重點]
了解 F1-score 的公式意義，並試著理解程式碼

## 作業
請參考 F1-score 的公式與[原始碼](https://github.com/scikit-learn/scikit-learn/blob/bac89c2/sklearn/metrics/classification.py#L620)，試著寫出 F2-Score 的計算函數

In [43]:
idx = np.arange(0,len(y_test),1)

true_pos = len(idx[(y_pred_binarized == 1)&(y_test == 1)])
false_neg = len(idx[(y_pred_binarized == 0)&(y_test == 1)])
false_pos = len(idx[(y_pred_binarized == 1)&(y_test == 0)])
def f_beta(beta, tp, fn, fp):
    return (1 + beta**2) * tp / ((1 + beta**2) * tp + beta**2 * fn + fp)
f2_score = f_beta(1, true_pos, false_neg, false_pos)
print(f2_score)

0.5098039215686274


In [44]:
true_pos/(true_pos+false_pos)

0.65

In [42]:
false_neg

18